In [1]:
# Tool to craw all files with a specific extension from a directory and subdirectories

# Load dependancies
from tkinter import filedialog
from tkinter import * # File dialog
from tkinter import messagebox as tkMessageBox # Error box (for WIN)
import os
import subprocess # Error box (for MacOS)
from czifile import CziFile # Import .czi files
import matplotlib.pyplot as plt # For plotting
from matplotlib.colors import LinearSegmentedColormap # Create own colormap
import numpy
import numpy as np
from shutil import copyfile # Copy file
import cv2

In [2]:
# Determine operation system
# 'posix' = MacOS
# 'nt' = WIN
operation_sys = os.name

In [3]:
# User input
output_ext = '.png'


# COLOR MAP
# Define Color for merged label
bg_col = tuple([x/255 for x in (64, 224, 208)])
obj_col = tuple([x/255 for x in (255, 165, 0)])
bound_col = tuple([x/255 for x in (255, 0, 0)])

# Define Color
colors = [obj_col, bg_col, bound_col]  # R -> G -> B
n_bins = [1, 2, 3]  # Discretizes the interpolation into bins
cmap_name = 'my_list'
# Create the colormap
cm = LinearSegmentedColormap.from_list(cmap_name, colors)
# Fewer bins will result in "coarser" colomap interpolation
#plt.imshow(rgb_label, interpolation='nearest', origin='lower', cmap=cm)
    
# Maybe have a list of extensions grouped
# Check if input and label file has the same number of images

In [4]:
# ALL DEFINED FUNCTIONS


# Read out all directories and files
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

if operation_sys == 'nt':
    # Error box (for WIN)
    def clicked():
        tkMessageBox.showerror("Error", "Unequal number of input images and label.")
elif operation_sys == 'posix':
    applescript = """
    display dialog "Unequal number of input images and label." ¬
    with title "Error" ¬
    with icon caution ¬
    buttons {"OK"}
    """

In [5]:
# Get user-specific folder selection
root = Tk()
root.withdraw()
folder_selected = filedialog.askdirectory()
# root.destroy() # Destroy root window

In [6]:
# Get sub dir
dir_sub = next(os.walk(folder_selected))[1]
# Construct path
dir_input = os.path.join( folder_selected, dir_sub[0] )
dir_label = os.path.join( folder_selected, dir_sub[1] )

In [7]:
# Read all files in sub dir (ignore sub-sub dir)
dir_list_input = getListOfFiles(dir_input)
dir_list_label = getListOfFiles(dir_label)

In [9]:
# Check if number of input and label images are equal
n_img = len(dir_list_input)
n_label = len(dir_list_label)

if n_img != n_label:
    if operation_sys == 'nt':
        root = Tk()
        button = Button(root, text = "show message box", command = clicked)
        button.pack()
        root.mainloop()
    elif operation_sys == 'posix':
        subprocess.call("osascript -e '{}'".format(applescript), shell=True)
else:
    pass

In [1]:
# Define export dir
# Get user-specific folder selection
root = Tk()
root.withdraw()
folder_selected_output = filedialog.askdirectory()
# root.destroy() # Destroy root window

NameError: name 'Tk' is not defined

In [11]:
# Define Folder structure
# Output folder
outFolder = 'Crawler'
# Images
imgFolder = 'Images'
# Labels_rgb
labelFolderRGB = 'Labels_rgb'
# Labels
labelFolder = 'Labels'

In [12]:
# ------------------------------------------------------------------------------------------ #
# Check for Folder

# Output
if os.path.exists( os.path.join(folder_selected_output, outFolder) ):
    print('Output: ##### Output folder is available. #####')
else:
    os.makedirs(os.path.join(folder_selected_output, outFolder))
    print('Output: ##### Create output folder called Crawler. #####')

# Image folder
if os.path.exists( os.path.join(folder_selected_output, outFolder, imgFolder) ):
    print('Output: ##### Image folder is available. #####')
else:
    os.makedirs( os.path.join(folder_selected_output, outFolder, imgFolder) )
    print('Output: ##### Create Image folder. #####')
    
# Label RGB folder
if os.path.exists( os.path.join(folder_selected_output, outFolder, labelFolderRGB) ):
    print('Output: ##### Label RGB folder is available. #####')
else:
    os.makedirs( os.path.join(folder_selected_output, outFolder, labelFolderRGB) )
    print('Output: ##### Create Label RGB folder. #####')
    
# Label folder
if os.path.exists( os.path.join(folder_selected_output, outFolder, labelFolder) ):
    print('Output: ##### Label folder is available. #####')
else:
    os.makedirs( os.path.join(folder_selected_output, outFolder, labelFolder) )
    print('Output: ##### Create Label folder. #####')
# ------------------------------------------------------------------------------------------ #


Output: ##### Create output folder called Crawler. #####
Output: ##### Create Image folder. #####
Output: ##### Create Label RGB folder. #####
Output: ##### Create Label folder. #####


In [13]:
for ii in range( 0, n_img ):
    ### Check input images
    # Obtain filename
    th_fname_input = os.path.basename(dir_list_input[ii])
    # Obtain case-insensitive extension
    if th_fname_input.lower().endswith('.czi'):
        # Load czi file
        with CziFile(dir_list_input[ii]) as czi: img_input = czi.asarray()
        # Remove 1D array
        img_input = numpy.squeeze(img_input)
        # Get only filename without path or extension
        fname_only_input = os.path.splitext(th_fname_input)[0]
        # Save image to .png
        cv2.imwrite(os.path.join(folder_selected_output, outFolder, imgFolder, fname_only + output_ext), img_input)
    else:
        copyfile( dir_list_input[ii], os.path.join(folder_selected_output, outFolder, imgFolder, th_fname_input) )
    
    ### Check labels
    
    # Obtain filename
    th_fname_label = os.path.basename(dir_list_label[ii])
    # Obtain case-insensitive extension
    if th_fname_label.lower().endswith('.czi'):
        # Load czi file`
        with CziFile(dir_list_label[ii]) as czi: img_label = czi.asarray()
        # Remove 1D array
        img_label = numpy.squeeze(img_label)
        # Get only filename without path or extension
        fname_only_label = os.path.splitext(th_fname_label)[0]
        # Save label to .png with 2 different format
        # 1. RGB, 2. Keep annotation e.g. 0 = Background, 1 = Foreground, 2 = Object
        cv2.imwrite(os.path.join(folder_selected_output, outFolder, labelFolder, fname_only_label + output_ext), img_label)        
        # Convert label to RGB 
        plt.imsave(os.path.join(folder_selected_output, outFolder, labelFolderRGB, fname_only_label + output_ext), img_label, cmap=cm)
    else:
        copyfile( dir_list_label[ii], os.path.join(folder_selected_output, outFolder, labelFolder, th_fname_label) )
